#### Importações

In [25]:
import pandas as pd
import matplotlib.pyplot as plt

# Configurações para exibição do DataFrame
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
pd.set_option('display.max_rows', None)     # Mostrar todas as linhas
pd.set_option('display.max_colwidth', None) # Mostrar todo o conteúdo das células

#### Importando os dataframes que serão analisados

In [26]:
df_original = pd.read_csv("../dados/nova_plataforma.csv")
df_gan = pd.read_csv("../dados/registros_gan.csv")

##### Removendo as colunas de df_original que não estão na df_gan e df_regessao

In [27]:
colunas_para_remover = ['platform', 'recommended (1 partial; 2 complete)', 'impacts_of_option-partial']
df_original.drop(colunas_para_remover, axis=1, inplace=True)

#### Analisando os registros gerados pela gan

##### Concatenado as estatisticas obtidas do df_original e df_gan

In [28]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [29]:
original_x_gan.head(25)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,126.268200,22.307377,21.814684,99.000,99.000,107.500,1.030000e+02,1.160000e+02,1.250000e+02,142.500000,1.500000e+02,1.560000e+02,1.550000e+02
weight (t),18815.714286,18086.908900,5472.264453,3663.086319,10054.000,10163.000,16530.500,1.533875e+04,1.858400e+04,1.794700e+04,21182.000000,2.071525e+04,2.764700e+04,2.748900e+04
installation_date,38.857143,39.071300,5.639993,4.955065,31.000,31.000,34.500,3.400000e+01,4.100000e+01,4.000000e+01,42.500000,4.400000e+01,4.600000e+01,4.600000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.517600,0.786796,0.805081,1.000,1.000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,2.000000e+00,3.000000e+00,3.000000e+00
number_of_legs,7.428571,6.589200,1.511858,0.955581,4.000,4.000,8.000,7.000000e+00,8.000000e+00,7.000000e+00,8.000000,7.000000e+00,8.000000e+00,8.000000e+00
number_of_piles,17.142857,20.876600,10.106575,9.824410,5.000,5.000,8.500,1.100000e+01,2.000000e+01,2.400000e+01,23.000000,3.100000e+01,3.200000e+01,3.100000e+01
height_of_jacket_or_sub-structure (m),137.500000,135.338387,18.993420,18.186993,114.000,114.000,122.750,1.169080e+02,1.360000e+02,1.319100e+02,150.500000,1.534110e+02,1.660000e+02,1.659520e+02
distance_to_coast (km),218.571429,232.069800,66.271807,57.103583,120.000,120.000,180.000,1.830000e+02,2.400000e+02,2.630000e+02,264.000000,2.800000e+02,2.820000e+02,2.820000e+02
risk_to_other_users-complete,0.428571,0.467500,0.534522,0.498968,0.000,0.000,0.000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,0.000005,0.000009,0.000009,0.000,0.000,0.000,4.426315e-10,7.800000e-08,7.717074e-08,0.000012,8.983989e-06,2.300000e-05,2.300000e-05


#### Removendo registros que contém células que ultrapassam o intervalo da coluna (min e max)

In [30]:
def ultrapassou_intervalo(registro, coluna, df_original):
    if registro > df_original[coluna].max() or registro < df_original[coluna].min():
        return True
    else:
        return False

In [31]:
def remover_registros_fora_do_intervalo(df_gan, df_original):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            if ultrapassou_intervalo(linha[coluna], coluna, df_original):
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro
    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [32]:
df_gan_limpo = remover_registros_fora_do_intervalo(df_gan, df_original)

0


In [33]:
df_gan_limpo.shape

(10000, 25)

#### Removendo outliers com base na amplitude interquartil

In [34]:
def remover_outliers_iqr(df_gan):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            q1 = df_gan[coluna].quantile(0.25)
            q3 = df_gan[coluna].quantile(0.75)
            
            iqr = q3 - q1 # Amplitude interquantil
            limite_superior = q3 + 1.5 * iqr
            limite_inferior = q1 - 1.5 * iqr

            if linha[coluna] < limite_inferior or linha[coluna] > limite_superior:
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro

    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [35]:
df_gan_limpo = remover_outliers_iqr(df_gan)

5900


In [36]:
df_gan.shape, df_gan_limpo.shape

((10000, 25), (4100, 25))

In [37]:
df_gan_limpo.duplicated().sum()

0

#### Comparando novamente as bases de dados

In [38]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan_limpo.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [39]:
original_x_gan.head(26)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,112.653415,22.307377,16.747107,99.000,99.000,107.500,1.000000e+02,1.160000e+02,1.040000e+02,142.500000,1.220000e+02,1.560000e+02,1.550000e+02
weight (t),18815.714286,19635.317317,5472.264453,3054.329230,10054.000,11506.000,16530.500,1.736000e+04,1.858400e+04,1.946200e+04,21182.000000,2.184475e+04,2.764700e+04,2.748900e+04
installation_date,38.857143,39.090976,5.639993,3.816448,31.000,31.000,34.500,3.600000e+01,4.100000e+01,3.900000e+01,42.500000,4.200000e+01,4.600000e+01,4.500000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.154146,0.786796,0.461914,1.000,1.000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,1.000000e+00,3.000000e+00,3.000000e+00
number_of_legs,7.428571,7.000000,1.511858,0.000000,4.000,7.000,8.000,7.000000e+00,8.000000e+00,7.000000e+00,8.000000,7.000000e+00,8.000000e+00,7.000000e+00
number_of_piles,17.142857,15.326585,10.106575,8.723445,5.000,5.000,8.500,7.000000e+00,2.000000e+01,1.400000e+01,23.000000,2.300000e+01,3.200000e+01,3.100000e+01
height_of_jacket_or_sub-structure (m),137.500000,122.868319,18.993420,12.123863,114.000,114.000,122.750,1.144603e+02,1.360000e+02,1.166345e+02,150.500000,1.268858e+02,1.660000e+02,1.640280e+02
distance_to_coast (km),218.571429,251.465610,66.271807,38.678109,120.000,123.000,180.000,2.350000e+02,2.400000e+02,2.710000e+02,264.000000,2.800000e+02,2.820000e+02,2.810000e+02
risk_to_other_users-complete,0.428571,0.750732,0.534522,0.432642,0.000,0.000,0.000,1.000000e+00,0.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,0.000001,0.000009,0.000004,0.000,0.000,0.000,8.225441e-12,7.800000e-08,1.285568e-09,0.000012,1.814527e-07,2.300000e-05,2.227400e-05


In [40]:
describe_diferenca = (df_original.describe() - df_gan.describe()).transpose()
describe_diferenca.drop(columns='count', inplace=True)
describe_diferenca

,mean,std,min,25%,50%,75%,max
water_depth (m),-1.839629,4.926921e-01,0.0,4.500000e+00,-9.000000e+00,-7.500000,1.000000e+00
weight (t),728.805386,1.809178e+03,-109.0,1.191750e+03,6.370000e+02,466.750000,1.580000e+02
installation_date,-0.214157,6.849280e-01,0.0,5.000000e-01,1.000000e+00,-1.500000,0.000000e+00
type_of_production (1 oil and gas; 2 oil; 3 gas),0.053829,-1.828498e-02,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
number_of_legs,0.839371,5.562771e-01,0.0,1.000000e+00,1.000000e+00,1.000000,0.000000e+00
number_of_piles,-3.733743,2.821653e-01,0.0,-2.500000e+00,-4.000000e+00,-8.000000,1.000000e+00
height_of_jacket_or_sub-structure (m),2.161613,8.064272e-01,0.0,5.842000e+00,4.090000e+00,-2.911000,4.800000e-02
distance_to_coast (km),-13.498371,9.168224e+00,0.0,-3.000000e+00,-2.300000e+01,-16.000000,0.000000e+00
risk_to_other_users-complete,-0.038929,3.555490e-02,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
risk_to_other_users-partial,0.000001,6.600109e-07,0.0,-4.426315e-10,8.292555e-10,0.000003,1.370907e-12


#### Analisando a semelhança dos dataframe com base no KSComplement e CorrelationSimilarity

In [41]:
from sdmetrics.reports.single_table import QualityReport
from sdv.metadata import SingleTableMetadata

In [42]:
df_original.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
0,99,22000,36,1,8,8,114.0,269,1,0.000000e+00,0.290,0.12,1110100,817000,87000,58500,0.00,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
1,112,20364,41,1,8,9,123.0,259,1,0.000000e+00,0.320,0.16,1180500,895500,92000,64000,0.00,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
2,141,15561,46,2,8,26,147.0,120,0,2.300000e-05,0.025,0.01,297654,530148,24277,31064,0.66,0.25,1.00,1.00,0.94,1.0,1.0,0.53,1.00
3,156,27647,44,2,8,32,166.0,240,0,1.500000e-05,0.040,0.02,487750,570818,40416,45266,1.00,0.50,1.00,1.00,0.66,1.0,1.0,1.00,0.57
4,103,18584,33,1,8,20,122.5,230,0,7.800000e-08,0.090,0.06,733082,511765,59588,41170,0.00,0.39,0.15,0.00,0.00,0.0,0.0,1.00,0.73


In [43]:
df_gan_limpo.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
1,103,20227,43,1,7,8,114.843,239,1,1.182407e-09,0.309,0.153,1127738,870174,86399,62709,0.000,0.000,1.000,0.724,0.894,0.162,0.640,0.987,0.727
4,99,18245,32,1,7,8,114.434,281,1,2.056360e-12,0.187,0.074,1053682,702538,84806,56509,0.000,0.000,1.000,0.997,0.931,0.138,0.564,1.000,0.491
8,108,21174,34,1,7,25,117.585,274,1,4.652172e-09,0.208,0.080,1048972,620747,75273,58064,0.000,0.005,0.615,0.465,0.096,0.029,0.221,0.999,0.692
9,103,17631,36,1,7,7,116.373,279,1,9.424984e-10,0.229,0.123,983555,728624,79155,54304,0.000,0.000,0.999,0.969,0.905,0.328,0.529,0.999,0.518
10,129,14604,40,1,7,21,129.072,222,0,2.382568e-07,0.175,0.122,805516,565252,54933,46092,0.001,0.018,0.859,0.297,0.232,0.078,0.181,0.944,0.687


In [50]:
# Supondo que A e B são seus DataFrames reais e sintéticos
A = df_original.copy()
B = df_gan.copy()

# Criar o metadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=A)
metadata_dict = metadata.to_dict()

# Gerar o relatório de qualidade
report = QualityReport()
report.generate(real_data=A, synthetic_data=B, metadata=metadata_dict)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 25/25 [00:00<00:00, 459.78it/s]|
Column Shapes Score: 69.89%

(2/2) Evaluating Column Pair Trends: |██████████| 300/300 [00:00<00:00, 345.37it/s]|
Column Pair Trends Score: 90.89%

Overall Score (Average): 80.39%



In [45]:
report.get_score()

0.8038970632308053

In [46]:
report.get_properties()

,Property,Score
0,Column Shapes,0.698903
1,Column Pair Trends,0.908891


In [47]:
report.get_details(property_name='Column Shapes')

,Column,Metric,Score
0,installation_date,KSComplement,0.857243
1,type_of_production (1 oil and gas; 2 oil; 3 gas),KSComplement,0.889829
2,number_of_legs,KSComplement,0.158657
3,number_of_piles,KSComplement,0.699357
4,height_of_jacket_or_sub-structure (m),KSComplement,0.768657
5,risk_to_other_users-complete,KSComplement,0.961071
6,risk_to_other_users-partial,KSComplement,0.640529
7,risk_to_personnel-complete,KSComplement,0.722214
8,risk_to_personnel-partial,KSComplement,0.753686
9,impacts_of_option-complete,KSComplement,0.708514


In [48]:
report.get_details(property_name='Column Pair Trends')

,Column 1,Column 2,Metric,Score,Real Correlation,Synthetic Correlation
0,installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),CorrelationSimilarity,0.978508,0.622398,0.665382
1,installation_date,number_of_legs,CorrelationSimilarity,0.936452,0.614305,0.487210
2,installation_date,number_of_piles,CorrelationSimilarity,0.872851,0.681692,0.427394
3,installation_date,height_of_jacket_or_sub-structure (m),CorrelationSimilarity,0.986227,0.599779,0.572232
4,installation_date,risk_to_other_users-complete,CorrelationSimilarity,0.993882,-0.473868,-0.461632
5,installation_date,risk_to_other_users-partial,CorrelationSimilarity,0.926292,0.826764,0.679348
6,installation_date,risk_to_personnel-complete,CorrelationSimilarity,0.780571,-0.277817,0.161041
7,installation_date,risk_to_personnel-partial,CorrelationSimilarity,0.729524,-0.274356,0.266596
8,installation_date,impacts_of_option-complete,CorrelationSimilarity,0.885484,0.698996,0.469965
9,installation_date,technical_feasibility_or_challenge-complete,CorrelationSimilarity,0.990838,0.397458,0.379134
